### midas_active_accounts_prd

Comando 1,2 e 3 igauis ao notebook no Databricks

In [1]:
query = f'''
select distinct
        date_format(t.datageracao, 'y-MM') as year_month,
        min(id_transacao) over (partition by t.id_conta, date_format(t.datageracao, 'y-MM'),t.issuerpartition) as id_transacao,
        t.issuerpartition as issuer_id,
        t.id_conta as account_id
    from cleaned_issuing.transacoescorrentes t
        left join cleaned_issuing.tipostransacoes tt
            on t.issuerpartition = tt.issuerpartition
            and t.id_tipotransacao = tt.id_tipotransacao
    where
        t.datageracao >= '2021-01-01' 
        and tt.flagcredito = 'false' 
        and t.id_conta not in (1,2,3,4,5) 
        and tt.flagextrato = 'true'
        
    '''
spark.sql(query).createOrReplaceTempView('active_account_midas')
 


Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
914,application_1653433423608_0988,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
query = f'''
select t.*, tt.descricao
from cleaned_issuing.transacoescorrentes t
left join cleaned_issuing.tipostransacoes tt  on t.issuerpartition = tt.issuerpartition and t.id_tipotransacao = tt.id_tipotransacao
 '''
spark.sql(query).createOrReplaceTempView('comp_info')


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
query = f'''
select a.year_month, a.issuer_id, a.account_id, t.id_tipotransacao, t.descricao, a.id_transacao, t.datageracao 
from active_account_midas a
inner join comp_info t on a.issuer_id = t.issuerpartition and a.id_transacao = t.id_transacao
'''
##spark.sql(query).write.mode("overwrite").saveAsTable("lakehouse.bt_midas_activeaccount", overwrite=True, format='parquet', path='s3://dock-datalake-prd/lakehouse/bt_midas_activeaccount')
query = spark.sql(query)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Como é feito no Databricks

In [ ]:
scope = 'databankstatements-prd'
host = dbutils.secrets.get(scope, "host")
port = dbutils.secrets.get(scope, "port")
database = dbutils.secrets.get(scope, "database")
username = dbutils.secrets.get(scope, "username")
password = dbutils.secrets.get(scope, "password")
output_table=f"midas.bt_active_account"

query\
    .write.options(truncate='true')\
    .format('jdbc')\
    .mode('overwrite')\
    .options(url=f'jdbc:postgresql://{host}:{port}/{database}?rewriteBatchedStatements=true', dbtable=output_table, user=username, password=password).save()

### Adaptação que a Camis me ajudou 

In [6]:
from pyspark.sql.window import *
from pyspark.sql.functions import *
from pyspark.sql.types import StringType
from pyspark.storagelevel import StorageLevel
from pyspark.sql.types import IntegerType
from pyspark.sql import SparkSession
import numpy as np
import pandas as pd
import boto3
import json
import psycopg2

spark.conf.set("spark.sql.files.ignoreCorruptFiles", "true")
spark.conf.set("spark.sql.orc.enabled","true")
spark.conf.set("spark.sql.sources.partitionOverwriteMode","dynamic")
spark.conf.set("spark.sql.sources.partitionColumnTypeInference.enabled", "false")
spark.conf.set("spark.databricks.io.cache.enabled", "true")
spark.conf.set("spark.databricks.delta.autoCompact.enabled", "true")
spark.conf.set("spark.databricks.delta.optimizeWrite.enabled", "true")
spark.conf.set("spark.databricks.delta.retentionDurationCheck.enabled", "false")

client = boto3.client('secretsmanager',"us-east-1")
response = client.get_secret_value(
    SecretId='Databricks_DatabankstatementsPrdDbPassword'
)

connectionProperties = {\
                        "user":json.loads(response['SecretString'])['username'],\
                        "password":json.loads(response['SecretString'])['password'],\
                        "host":json.loads(response['SecretString'])['host'],\
                        "dbname":json.loads(response['SecretString'])['dbname']\
                       }

host = json.loads(response['SecretString'])['host']
port = 5439
database = json.loads(response['SecretString'])['dbname']
username = json.loads(response['SecretString'])['username']
password = json.loads(response['SecretString'])['password']
materialized_view = 'midas.bt_active_account'

con=psycopg2.connect(
  dbname= database,
  host=host, 
  port=port,
  user=username,
  password=password)

query\
    .write.options(truncate='true')\
    .format('jdbc')\
    .mode('overwrite')\
    .options(url=f'jdbc:postgresql://{host}:{port}/{database}?rewriteBatchedStatements=true', dbtable=output_table, user=username, password=password).save()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
could not connect to server: Connection timed out
	Is the server running on host "data-bank-statements.cw1dtrln74e9.sa-east-1.rds.amazonaws.com" (10.140.3.82) and accepting
	TCP/IP connections on port 5439?

Traceback (most recent call last):
  File "/usr/local/lib64/python3.7/site-packages/psycopg2/__init__.py", line 122, in connect
    conn = _connect(dsn, connection_factory=connection_factory, **kwasync)
psycopg2.OperationalError: could not connect to server: Connection timed out
	Is the server running on host "data-bank-statements.cw1dtrln74e9.sa-east-1.rds.amazonaws.com" (10.140.3.82) and accepting
	TCP/IP connections on port 5439?


